# 📦 Importações

In [1]:
! pip install pandas numpy statsmodels statsforecast scikit-learn tensorflow yfinance

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB 1.4 MB/s eta 0:00:03
      --------------------------------------- 0.0/3.0 MB 393.8 kB/s eta 0:00:08
      --------------------------------------- 0.1/3.0 MB 660.6 kB/s eta 0:00:05
     - -------------------------------------- 0.1/3.0 MB 778.5 kB/s eta 0:00:04
     -- ------------------------------------- 0.2/3.0 MB 657.6 kB/s eta 0:00:05
     -- ------------------------------------- 0.2/3.0 MB 778.2 kB/s eta 0:00:04
     -- ------------------------------------- 0.2/3.0 MB 778.2 kB/s eta 0:00:04
     ---- ----------------------------------- 0.3/3.0 MB 985.2 kB/s eta 0:00:03
     ------ --------------------------------- 0.5/3.0 MB 1.3 MB/s eta 0:00:03
     -------- ------------------------------- 0.6/3.0 MB 1.4 MB/s eta 0:00

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.4 which is incompatible.


In [2]:
import os
import sqlite3
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM
import plotly.graph_objects as go

# 📂 Funções de Banco de Dados (SQLite)

In [3]:
def criar_banco():
    conn = sqlite3.connect('finance.db')
    cur = conn.cursor()
    cur.execute('''
        CREATE TABLE IF NOT EXISTS historico (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            ativo TEXT,
            data DATE,
            fechamento REAL
        )
    ''')
    conn.commit()
    conn.close()

def inserir_dados_historicos(ativo, dataframe):
    conn = sqlite3.connect('finance.db')
    cur = conn.cursor()
    for index, row in dataframe.iterrows():
        date_str = row['Date'].strftime('%Y-%m-%d')
        cur.execute('''
            INSERT INTO historico (ativo, data, fechamento)
            VALUES (?, ?, ?)
        ''', (ativo, date_str, row['Close']))
    conn.commit()
    conn.close()

def carregar_dados_historicos(ativo):
    conn = sqlite3.connect('finance.db')
    df = pd.read_sql_query(f'''
        SELECT data, fechamento FROM historico WHERE ativo = '{ativo}' ORDER BY data
    ''', conn)
    conn.close()
    df['data'] = pd.to_datetime(df['data'])
    return df

# 📊 Funções de Modelagem (LSTM)

In [4]:
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

def predict_to_date(target_date, model, sequence_length, data_scaled, scaler, data):
    target_date = pd.to_datetime(target_date)
    last_known_date = pd.to_datetime(data['Date'].iloc[-1]).tz_localize(None)
    num_days = (target_date - last_known_date).days
    if num_days <= 0:
        raise ValueError("A data deve ser futura em relação aos dados atuais.")
    prediction_list = data_scaled[-sequence_length:]
    for _ in range(num_days):
        x = prediction_list[-sequence_length:]
        x = x.reshape((1, sequence_length, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = scaler.inverse_transform(prediction_list.reshape(-1, 1))
    return prediction_list[-1][0]

def gerar_dados_completos(num_prediction, model, sequence_length, data, data_scaled, scaler):
    prediction_list = data_scaled[-sequence_length:]
    for _ in range(num_prediction):
        x = prediction_list[-sequence_length:]
        x = x.reshape((1, sequence_length, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    forecast_full = scaler.inverse_transform(prediction_list.reshape(-1, 1))
    forecast = forecast_full[-num_prediction:]
    last_date = data['Date'].iloc[-1]
    future_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()[1:]
    future_df = pd.DataFrame({ 'Date': future_dates, 'Close': forecast.flatten() })
    full_data = pd.concat([data[['Date', 'Close']], future_df], ignore_index=True)
    full_data['MM20'] = full_data['Close'].rolling(window=20).mean()
    full_data['MM80'] = full_data['Close'].rolling(window=80).mean()
    return full_data

def tomar_decisao(full_data, target_date):
    target_date = pd.to_datetime(target_date)
    full_data['Date_only'] = full_data['Date'].dt.date
    target_row = full_data[full_data['Date_only'] == target_date.date()]
    if target_row.empty:
        return "Data não encontrada."
    mm20 = target_row['MM20'].values[0]
    mm80 = target_row['MM80'].values[0]
    if np.isnan(mm20) or np.isnan(mm80):
        return "Não há dados suficientes para decisão."
    if mm20 > mm80:
        return "✅ Recomendacao: Comprar"
    else:
        return "⏳ Recomendacao: Aguardar"

def plotar_forecast(full_data, target_date):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=full_data['Date'], y=full_data['Close'], mode='lines', name='Fechamento'))
    fig.add_trace(go.Scatter(x=full_data['Date'], y=full_data['MM20'], mode='lines', name='MM20'))
    fig.add_trace(go.Scatter(x=full_data['Date'], y=full_data['MM80'], mode='lines', name='MM80'))
    fig.add_shape(type="line", x0=pd.to_datetime(target_date), y0=full_data['Close'].min(),
                  x1=pd.to_datetime(target_date), y1=full_data['Close'].max(),
                  line=dict(color="Red", width=2, dash="dash"))
    fig.add_annotation(x=pd.to_datetime(target_date), y=full_data['Close'].max(),
                       text="Data escolhida", showarrow=True, arrowhead=1)
    fig.update_layout(title="Forecast + Medias Moveis", xaxis_title="Data", yaxis_title="Preco",
                      template="plotly_white", width=1000, height=600, xaxis=dict(type='date'))
    fig.show()

# 📅 Execução Principal

In [6]:
# Criar banco e inserir dados apenas se não existir
if not os.path.exists('finance.db'):
    criar_banco()
    ticker = 'BTC-USD'
    ticket = yf.Ticker(ticker)
    data = ticket.history(interval='1d', start='2023-01-20', end='2025-04-20')
    data.reset_index(inplace=True)
    inserir_dados_historicos(ticker, data[['Date', 'Close']])

# Carregar dados do banco
ticker = 'BTC-USD'
data = carregar_dados_historicos(ticker)
data.rename(columns={'data': 'Date', 'fechamento': 'Close'}, inplace=True)
data.reset_index(drop=True, inplace=True)

# Preparar dados
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[['Close']])
train_size = int(len(data_scaled) * 0.7)
train_data = data_scaled[:train_size]
test_data = data_scaled[train_size:]
sequence_length = 10
X_train, y_train = create_sequences(train_data, sequence_length)
X_test, y_test = create_sequences(test_data, sequence_length)

# Verificar se modelo existe
if os.path.exists('modelo_lstm.h5'):
    model_lstm = load_model('modelo_lstm.h5', compile=False)
else:
    model_lstm = Sequential([
        LSTM(64, input_shape=(sequence_length, 1)),
        Dense(1)
    ])
    model_lstm.compile(optimizer='adam', loss='mse')
    model_lstm.fit(X_train, y_train, epochs=50, batch_size=32)
    model_lstm.save('modelo_lstm.h5')

# Avaliar modelo
lstm_predictions = model_lstm.predict(X_test)
print()
print(f"R2: {r2_score(y_test, lstm_predictions):.4f} | MAPE: {np.mean(np.abs((y_test - lstm_predictions) / y_test)) * 100:.2f}%")

Epoch 1/50


c:\Users\andre\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0537
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0050 
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0014
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0010 
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.1629e-04
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.1762e-04
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.3680e-04 
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.6779e-04
Epoch 9/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.0235e-04 
Epoch 10/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.2512e-04
Epoch 11/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.5423e-04
Epoch 12/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.8942e-04
Epoch 13/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.0970e-04
Epoch 14/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.4667e-04
Epoch 15/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step -

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

R2: 0.9142 | MAPE: 5.45%


# 💰 Previsão

In [7]:
target_date = '2025-04-25'
preco_previsto = predict_to_date(target_date, model_lstm, sequence_length, data_scaled, scaler, data)
full_data = gerar_dados_completos(30, model_lstm, sequence_length, data, data_scaled, scaler)
plotar_forecast(full_data, target_date)
recomendacao = tomar_decisao(full_data, target_date)

print(f"📅 Data: {target_date}")
print(f"💰 Preço Previsto: R$ {preco_previsto:.2f}")
print(f"🧠 {recomendacao}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━

📅 Data: 2025-04-25
💰 Preço Previsto: R$ 79374.97
🧠 ⏳ Recomendacao: Aguardar
